In [13]:
import os,sys
import os.path
currentdir = '/home/xyao/11777-Project-VRD/MFURLN-CVPR/test'
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
import tensorflow as tf 
import numpy as np 
import json
from collections import defaultdict

from model.config import cfg 
from model.ass_fun import *
from net.vrd_rela import MFURLN

In [51]:
import pickle


with open('/data/xyao/sg_dataset/MFURLN/output/result.pkl', 'rb') as f:
    data = pickle.load(f)
data

{'3845770407_1a8cd41230_b.jpg': defaultdict(list,
             {'woman_192275885568_earring_285477309497': [('is wearing', 1.0)],
              'streetlamp_3112377165_building_424307223': [('in front of',
                1.0)],
              'vehicle_332529454750_tree_10274440747': [('is behind', 1.0)],
              'frisbee_412217464360_grass_42161019744': [('is above', 1.0)],
              'woman_192275885568_sneakers_747275893481': [('wears', 1.0)],
              'balloon_292193314218_building_424307223': [('in front of',
                1.0)],
              'woman_192275885568_shorts_465356592566': [('wears', 1.0)],
              'tree_10274440747_woman_192275885568': [('is behind', 1.0)],
              'tree_10274440747_vehicle_332529454750': [('is in front of',
                1.0)],
              'woman_192275885568_tree_10274440747': [('in front of', 1.0)]}),
 '6279585054_1f9d450f35_b.jpg': defaultdict(list,
             {'boy_108369682686_eye_186505205525': [('has', 1.0)],
  

In [14]:
predict_roidb = read_roidb('/data/xyao/sg_dataset/MFURLN/output/pred/vrd_roid_predicate.npz')
gt_roidb = read_roidb('/data/xyao/sg_dataset/MFURLN/process/vrd_pred_process_roidb.npz')['test_roidb']
with open('/data/xyao/sg_dataset/json_dataset/annotations_test.json', 'r') as f:
    test_raw = json.load(f)
with open('/data/xyao/sg_dataset/json_dataset/objects.json', 'r') as f:
    objects_raw = json.load(f)
with open('/data/xyao/sg_dataset/json_dataset/predicates.json', 'r') as f:
    predicates_raw = json.load(f)

In [7]:
print(len(predict_roidb['pred_roidb']), len(gt_roidb))
print(predict_roidb['pred_roidb'][0].keys(), gt_roidb[0].keys())

954 954
dict_keys(['pred_rela', 'pred_rela_score', 'pred_rela_all', 'sub_box_dete', 'obj_box_dete', 'sub_dete', 'obj_dete']) dict_keys(['image', 'width', 'height', 'index_pred', 'sub_box_gt', 'obj_box_gt', 'sub_gt', 'obj_gt', 'rela_gt'])


In [10]:
test_raw[repeat_imgs[2]]

[{'predicate': 3,
  'object': {'category': 49, 'bbox': [495, 702, 67, 178]},
  'subject': {'category': 49, 'bbox': [493, 828, 192, 312]}},
 {'predicate': 0,
  'object': {'category': 5, 'bbox': [1, 955, 2, 1277]},
  'subject': {'category': 71, 'bbox': [297, 455, 60, 608]}},
 {'predicate': 9,
  'object': {'category': 5, 'bbox': [1, 955, 2, 1277]},
  'subject': {'category': 71, 'bbox': [297, 455, 60, 608]}},
 {'predicate': 23,
  'object': {'category': 71, 'bbox': [297, 455, 60, 608]},
  'subject': {'category': 5, 'bbox': [1, 955, 2, 1277]}},
 {'predicate': 29,
  'object': {'category': 71, 'bbox': [297, 455, 60, 608]},
  'subject': {'category': 5, 'bbox': [1, 955, 2, 1277]}}]

In [6]:
repeat_imgs = []
for image_name in list(test_raw.keys()):
    sample = test_raw[image_name]
    sub_obj_pair = []
    for pair in sample:
        sub = pair['subject']['category']
        obj = pair['object']['category']
        pred = predicates_raw[pair['predicate']]
        sub_pos = str(pair['subject']['bbox'][0])+'/'+str(pair['subject']['bbox'][1])+'/'+str(pair['subject']['bbox'][2])+'/'+str(pair['subject']['bbox'][-1])
        obj_pos = str(pair['object']['bbox'][0])+'/'+str(pair['object']['bbox'][1])+'/'+str(pair['object']['bbox'][2])+'/'+str(pair['object']['bbox'][-1])
        sub_obj_pair.append(objects_raw[sub]+'_'+sub_pos+'_'+obj_pos+'_'+objects_raw[obj])
    if len(sub_obj_pair) != len(set(sub_obj_pair)):
        repeat_imgs.append(image_name)
        print(image_name, sub_obj_pair)

8646018805_d914413321_b.jpg ['person_145/536/157/494_145/199/316/392_helmet', 'person_145/536/157/494_174/323/322/467_shirt', 'person_145/536/157/494_427/478/157/203_shoe', 'person_145/536/157/494_420/537/158/491_shoes', 'person_145/536/157/494_287/513/185/491_pants', 'person_145/536/157/494_287/513/185/491_pants', 'person_145/536/157/494_145/199/316/392_helmet', 'pants_287/513/185/491_145/536/157/494_person', 'pants_287/513/185/491_145/199/316/392_helmet', 'pants_329/571/800/939_189/590/682/941_person', 'person_189/590/682/941_224/378/773/936_shirt', 'person_189/590/682/941_329/571/800/939_pants', 'person_301/578/534/777_352/512/611/770_shirt']
5870797431_00a5f8a60d_b.jpg ['motorcycle_292/630/6/598_259/655/12/1023_street', 'motorcycle_292/630/6/598_1/367/1/426_car', 'person_102/645/185/439_292/630/6/598_motorcycle', 'person_102/645/185/439_177/355/305/429_shirt', 'person_102/645/185/439_325/610/333/438_jeans', 'person_102/645/185/439_308/333/241/266_watch', 'car_1/367/1/426_259/655/12

In [43]:
test_raw[repeat_imgs[1]]

[{'predicate': 27,
  'object': {'category': 86, 'bbox': [208, 682, 656, 964]},
  'subject': {'category': 0, 'bbox': [228, 680, 500, 708]}},
 {'predicate': 27,
  'object': {'category': 86, 'bbox': [208, 682, 656, 964]},
  'subject': {'category': 0, 'bbox': [228, 680, 500, 708]}},
 {'predicate': 1,
  'object': {'category': 6, 'bbox': [354, 682, 454, 692]},
  'subject': {'category': 0, 'bbox': [228, 680, 500, 708]}},
 {'predicate': 1,
  'object': {'category': 88, 'bbox': [473, 681, 131, 198]},
  'subject': {'category': 0, 'bbox': [290, 679, 11, 358]}},
 {'predicate': 1,
  'object': {'category': 10, 'bbox': [362, 398, 89, 210]},
  'subject': {'category': 0, 'bbox': [290, 679, 11, 358]}},
 {'predicate': 1,
  'object': {'category': 10, 'bbox': [362, 398, 89, 210]},
  'subject': {'category': 0, 'bbox': [290, 679, 11, 358]}}]

In [21]:
print(predict_roidb['pred_roidb'][0].keys())
print('# of predict predicates', predict_roidb['pred_roidb'][0]['pred_rela'],predict_roidb['pred_roidb'][0]['pred_rela_score'], predict_roidb['pred_roidb'][0]['sub_dete'], predict_roidb['pred_roidb'][0]['obj_dete'])

print(gt_roidb[0].keys())
print('# of gt predicates', gt_roidb[0]['rela_gt'], gt_roidb[0]['sub_gt'], gt_roidb[0]['obj_gt'])

dict_keys(['pred_rela', 'pred_rela_score', 'pred_rela_all', 'sub_box_dete', 'obj_box_dete', 'sub_dete', 'obj_dete'])
# of predict predicates [ 9 23 23] [0.3895653  0.34725714 0.50689393] [25. 59. 48.] [59. 25. 25.]
dict_keys(['image', 'width', 'height', 'index_pred', 'sub_box_gt', 'obj_box_gt', 'sub_gt', 'obj_gt', 'rela_gt'])
# of gt predicates [ 0.  2. 14.] [25. 59. 48.] [59. 25. 25.]


In [31]:
print(gt_roidb[0].keys())
print(gt_roidb[0]['index_pred'], gt_roidb[0]['sub_box_gt'], gt_roidb[0]['obj_gt'], gt_roidb[0]['rela_gt'])

dict_keys(['image', 'width', 'height', 'index_pred', 'sub_box_gt', 'obj_box_gt', 'sub_gt', 'obj_gt', 'rela_gt'])
[0 1 2 2 2 2 2 1 0 2 2 1 2 2 0 2 0 2 0 2 0 2 2 1 1 2 2 2 1 0] [[788. 271. 873. 355.]
 [819. 357. 854. 481.]
 [  5. 537. 913. 718.]] [59. 25. 25.] [ 0.  2. 14.]


In [ ]:
predict_roidb = predict_roidb['pred_roidb']

In [55]:
def box_to_str(bbox):
    return '_'.join(map(str, map(int,bbox)))
box_to_str(sub_bbox_gt_list[j])

'489_182_528_213'

In [65]:
result = {}
assert len(gt_roidb) == len(predict_roidb)
N_data = len(gt_roidb)
for i in range(N_data):
    image_name = gt_roidb[i]['image'].split('/')[-1]
    gt_rela_list = gt_roidb[i]['rela_gt']
    N_rela = len(gt_rela_list)
    if N_rela == 0:
        continue
    result[image_name] = defaultdict(list)
    sub_gt_list = gt_roidb[i]['sub_gt']
    obj_gt_list = gt_roidb[i]['obj_gt']
    sub_bbox_gt_list = gt_roidb[i]['sub_box_gt']
    obj_bbox_gt_list = gt_roidb[i]['obj_box_gt']
    pred_rela_list = predict_roidb[i]['pred_rela']
    pred_rela_score_list = predict_roidb[i]['pred_rela_score']
    N_pred = len(pred_rela_list)
    assert N_pred == N_rela
    sort_score = -np.sort(-np.reshape(pred_rela_score_list, [1, -1]))
    if N_pred <= 50:
        thresh = -1
    else:
        thresh = sort_score[0][50]

    for j in range(N_pred):
        assert sub_gt_list[j] == predict_roidb[i]['sub_dete'][j]
        assert obj_gt_list[j] == predict_roidb[i]['obj_dete'][j]
        assert sub_bbox_gt_list[j].any() == predict_roidb[i]['sub_box_dete'][j].any()
        assert obj_bbox_gt_list[j].any() == predict_roidb[i]['obj_box_dete'][j].any()
        predict_predicate = predicates_raw[int(pred_rela_list[j])]
        predict_predicate_score = pred_rela_score_list[j]

        sub_bbox = box_to_str(sub_bbox_gt_list[j]) 
        obj_bbox = box_to_str(obj_bbox_gt_list[j]) 
        key_name = objects_raw[int(sub_gt_list[j])] + '_' + sub_bbox + '_' + objects_raw[int(obj_gt_list[j])] + '_' + obj_bbox
        result[image_name][key_name].append((predict_predicate,predict_predicate_score))





In [66]:
result

{'5531467160_2ae2cb76d4_b.jpg': defaultdict(list,
             {'clock_788_271_873_355_post_819_357_854_481': [('above',
                0.3895653)],
              'post_819_357_854_481_clock_788_271_873_355': [('below',
                0.34725714)],
              'bush_5_537_913_718_clock_788_271_873_355': [('below',
                0.50689393)]}),
 '8646018805_d914413321_b.jpg': defaultdict(list,
             {'person_157_145_494_536_helmet_316_145_392_199': [('wear',
                0.8180547),
               ('wear', 0.8180547)],
              'person_157_145_494_536_shirt_322_174_467_323': [('wear',
                0.7858735)],
              'person_157_145_494_536_shoe_157_427_203_478': [('has',
                0.47143108)],
              'person_157_145_494_536_shoes_158_420_491_537': [('wear',
                0.8010522)],
              'person_157_145_494_536_pants_185_287_491_513': [('wear',
                0.79707),
               ('wear', 0.79707)],
              'pants_185_

In [67]:
path_to_result = '/data/xyao/sg_dataset/MFURLN/output/mfurln_result.pkl'
with open(path_to_result, 'wb') as f:
    pickle.dump(result, f, protocol=pickle.HIGHEST_PROTOCOL)
    print("saved to %s"%(path_to_result))

saved to /data/xyao/sg_dataset/MFURLN/output/mfurln_result.pkl


In [77]:
gt_path = '/data/xyao/sg_dataset/MFURLN/output/mfurln_gt.pkl'
path_to_result = '/data/xyao/sg_dataset/MFURLN/output/mfurln_result.pkl'
preds = pickle.load(open(path_to_result, "rb"))
gt = pickle.load(open(gt_path, "rb"))

In [70]:
len(preds.keys())

954

In [69]:
len(gt.keys())

1000

In [78]:
print(list(preds.keys())[0], preds[list(preds.keys())[0]])
print(gt[list(preds.keys())[0]])

5531467160_2ae2cb76d4_b.jpg defaultdict(<class 'list'>, {'clock_271_355_788_873_post_357_481_819_854': [('above', 0.3895653)], 'post_357_481_819_854_clock_271_355_788_873': [('below', 0.34725714)], 'bush_537_718_5_913_clock_271_355_788_873': [('below', 0.50689393)]})
{'clock_271_355_788_873_post_357_481_819_854': [('on', 1.0)], 'post_357_481_819_854_clock_271_355_788_873': [('has', 1.0)], 'bush_537_718_5_913_clock_271_355_788_873': [('in the front of', 1.0)]}
